In [30]:
from numpy.random import RandomState
import pandas as pd
import jieba
import jieba.analyse
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer 

In [31]:
df = pd.read_csv('movies.csv')
#str -> dict
df['category'] = df['category'].apply(eval)    
def list_category(x):
    for first in x.values():
        return(first) 
df['main_category'] = df['category'].apply(list_category)

In [32]:
#split overview
def split_info(df):
    a = []
    for info in df['info']:
        keywords = jieba.analyse.extract_tags(info, topK=5)
        a.append(keywords)

    df['keyword'] = a
    return df['keyword']

In [33]:
df['keyword'] = split_info(df)

In [34]:
df['keywords'] = ''
def bag_words(x):
    return (''.join(re.sub('[^\u4e00-\u9fa5]+', '', x['ch_name'])) + ' ' + ''.join(x['main_category']) + ' ' +  ' '.join(x['keyword']))
df['keywords'] = df.apply(bag_words, axis = 1)

In [35]:
#split Data to train and test
rng = RandomState()
train = df.sample(frac=0.9173, random_state=rng)
test = df.loc[~df.index.isin(train.index)]
#test['主類別'].to_csv('test_Data.csv')

In [36]:
#轉詞頻矩陣
tfvectorizer=CountVectorizer() 
count_train=tfvectorizer.fit_transform(train['keywords'])# Tf 矩阵
count_test=tfvectorizer.transform(test['keywords']) 
transformer = TfidfTransformer() # 轉換Tf矩阵
tfidf_train = transformer.fit_transform(count_train) # 將TF轉換成Tf-Idf
tfidf_test = transformer.fit_transform(count_test)



In [37]:
knn  = KNeighborsClassifier(n_neighbors=11,p=2,n_jobs=2)
knn.fit(tfidf_train,train['main_category'])

KNeighborsClassifier(n_jobs=2, n_neighbors=11)

In [38]:
#predict test_Data
knn.predict(tfidf_test)

array(['動作', '動作', '動作', '劇情', '動作', '動作', '劇情', '動作', '動作', '動作', '動作',
       '動作', '動作', '動作', '動作', '劇情', '動作', '動作', '動作', '動作', '動作', '動作',
       '劇情', '劇情', '動作', '劇情', '動作', '動作', '動作', '動作', '動作', '動作', '劇情',
       '動作', '劇情', '動作', '劇情', '劇情', '劇情', '動作', '劇情', '動作', '動作', '劇情',
       '動作', '動作', '動作', '動作', '動作', '動作', '劇情', '動作', '動作', '劇情', '劇情',
       '動作', '動作', '動作', '動作', '動作', '動作', '劇情', '動作', '科幻', '劇情', '劇情',
       '劇情', '劇情', '動作', '動作', '動作', '愛情', '動作', '動作', '動作', '動作', '動作',
       '動作', '動作', '動作', '動作', '恐怖', '動作', '動作', '動作', '動作', '動作', '動作',
       '動作', '劇情', '動作', '動作', '劇情', '動作', '動作', '動作', '動作', '動作', '劇情',
       '劇情', '劇情', '劇情', '劇情', '動畫', '劇情', '劇情', '劇情', '劇情', '劇情', '劇情',
       '劇情', '劇情', '劇情', '劇情', '劇情', '愛情', '劇情', '劇情', '愛情', '歷史/傳記',
       '劇情', '劇情', '動畫', '劇情', '劇情', '劇情', '劇情', '動作', '劇情', '劇情', '科幻',
       '劇情', '劇情', '劇情', '劇情', '劇情', '劇情', '劇情', '劇情', '劇情', '劇情', '劇情',
       '劇情', '劇情', '劇情', '劇情', '動畫', '劇情', '劇情', '愛情',

In [39]:
#精準度%
Accuracy = knn.score(tfidf_test,test['main_category'])
print(Accuracy*100 , '%')

79.35153583617748 %
